# Using the cache

First make sure that the dependencies are installed:

```bash
pip install -r requirements-dev.txt
```

Then run redis using docker compose:

```bash
docker compose -f docker-compose.redis.yml up -d
```

Then run the notebook. 

If you want to view the data in redis, you can use the redis-cli:

```bash
brew install redis
redis-cli -p 6380
```

In [1]:
import fsspec
import xarray as xr
from redis import Redis
from redis_fsspec_cache.sync import RedisCachingFileSystem

redis = Redis(host="localhost", port=6380)

s3fs = fsspec.filesystem("s3", anon=True)
cached_s3fs = RedisCachingFileSystem(redis=redis, fs=s3fs, expiry_time=60, method='chunk')

Load a 321 mb file

In [2]:
%%time 

ds = xr.open_dataset(cached_s3fs.open('s3://noaa-nos-ofs-pds/cbofs/netcdf/202405/nos.cbofs.fields.f001.20240506.t00z.nc', 'rb'), chunks={}, engine='h5netcdf')
ds

CPU times: user 1.8 s, sys: 3.39 s, total: 5.19 s
Wall time: 18.1 s


<xarray.Dataset> Size: 61MB
Dimensions:         (tracer: 3, boundary: 4, s_rho: 20, s_w: 21, eta_rho: 291,
                     xi_rho: 332, eta_u: 291, xi_u: 331, eta_v: 290, xi_v: 332,
                     eta_psi: 290, xi_psi: 331, ocean_time: 1)
Coordinates:
  * s_rho           (s_rho) float64 160B -0.975 -0.925 -0.875 ... -0.075 -0.025
  * s_w             (s_w) float64 168B -1.0 -0.95 -0.9 -0.85 ... -0.1 -0.05 0.0
    lon_rho         (eta_rho, xi_rho) float64 773kB dask.array<chunksize=(291, 332), meta=np.ndarray>
    lat_rho         (eta_rho, xi_rho) float64 773kB dask.array<chunksize=(291, 332), meta=np.ndarray>
    lon_u           (eta_u, xi_u) float64 771kB dask.array<chunksize=(291, 331), meta=np.ndarray>
    lat_u           (eta_u, xi_u) float64 771kB dask.array<chunksize=(291, 331), meta=np.ndarray>
    lon_v           (eta_v, xi_v) float64 770kB dask.array<chunksize=(290, 332), meta=np.ndarray>
    lat_v           (eta_v, xi_v) float64 770kB dask.array<chunksize=(290, 332), meta=np.ndarray>
    lon_psi         (eta_psi, xi_psi) float64 768kB dask.array<chunksize=(290, 331), meta=np.ndarray>
    lat_psi         (eta_psi, xi_psi) float64 768kB dask.array<chunksize=(290, 331), meta=np.ndarray>
  * ocean_time      (ocean_time) datetime64[ns] 8B 2024-05-06T01:00:00
Dimensions without coordinates: tracer, boundary, eta_rho, xi_rho, eta_u, xi_u,
                                eta_v, xi_v, eta_psi, xi_psi
Data variables: (12/97)
    ntimes          int32 4B ...
    ndtfast         int32 4B ...
    dt              float64 8B ...
    dtfast          float64 8B ...
    dstart          datetime64[ns] 8B ...
    nHIS            int32 4B ...
    ...              ...
    temp            (ocean_time, s_rho, eta_rho, xi_rho) float32 8MB dask.array<chunksize=(1, 10, 146, 166), meta=np.ndarray>
    salt            (ocean_time, s_rho, eta_rho, xi_rho) float32 8MB dask.array<chunksize=(1, 10, 146, 166), meta=np.ndarray>
    oxygen          (ocean_time, s_rho, eta_rho, xi_rho) float32 8MB dask.array<chunksize=(1, 10, 146, 166), meta=np.ndarray>
    Pair            (ocean_time, eta_rho, xi_rho) float32 386kB dask.array<chunksize=(1, 291, 332), meta=np.ndarray>
    Uwind           (ocean_time, eta_rho, xi_rho) float32 386kB dask.array<chunksize=(1, 291, 332), meta=np.ndarray>
    Vwind           (ocean_time, eta_rho, xi_rho) float32 386kB dask.array<chunksize=(1, 291, 332), meta=np.ndarray>
Attributes: (12/36)
    file:              nos.cbofs.fields.forecast.20240506.t00z_0002.nc
    format:            netCDF-4/HDF5 file
    Conventions:       CF-1.4, SGRID-0.3
    type:              ROMS/TOMS history file
    title:             cbofs forecast RUN in operational mode
    var_info:          varinfo.yaml
    ...                ...
    compiler_flags:    -fp-model precise -ip -O3
    tiling:            008x016
    history:           ROMS/TOMS, Version 4.1, Monday - May 6, 2024 - 12:53:5...
    ana_file:          ROMS/Functionals/ana_btflux.h, ROMS/Functionals/ana_ra...
    CPP_options:       mode, ADD_FSOBC, ADD_M2OBC, ANA_BPFLUX, ANA_BSFLUX, AN...
    bio_file:          ROMS/Nonlinear/Biology/hypoxia_srm.h

In [3]:
%%time 

ds = xr.open_dataset(cached_s3fs.open('s3://noaa-nos-ofs-pds/cbofs/netcdf/202405/nos.cbofs.fields.f001.20240506.t00z.nc', 'rb'), chunks={}, engine='h5netcdf')
ds

CPU times: user 103 ms, sys: 8.68 ms, total: 112 ms
Wall time: 300 ms


<xarray.Dataset> Size: 61MB
Dimensions:         (tracer: 3, boundary: 4, s_rho: 20, s_w: 21, eta_rho: 291,
                     xi_rho: 332, eta_u: 291, xi_u: 331, eta_v: 290, xi_v: 332,
                     eta_psi: 290, xi_psi: 331, ocean_time: 1)
Coordinates:
  * s_rho           (s_rho) float64 160B -0.975 -0.925 -0.875 ... -0.075 -0.025
  * s_w             (s_w) float64 168B -1.0 -0.95 -0.9 -0.85 ... -0.1 -0.05 0.0
    lon_rho         (eta_rho, xi_rho) float64 773kB dask.array<chunksize=(291, 332), meta=np.ndarray>
    lat_rho         (eta_rho, xi_rho) float64 773kB dask.array<chunksize=(291, 332), meta=np.ndarray>
    lon_u           (eta_u, xi_u) float64 771kB dask.array<chunksize=(291, 331), meta=np.ndarray>
    lat_u           (eta_u, xi_u) float64 771kB dask.array<chunksize=(291, 331), meta=np.ndarray>
    lon_v           (eta_v, xi_v) float64 770kB dask.array<chunksize=(290, 332), meta=np.ndarray>
    lat_v           (eta_v, xi_v) float64 770kB dask.array<chunksize=(290, 332), meta=np.ndarray>
    lon_psi         (eta_psi, xi_psi) float64 768kB dask.array<chunksize=(290, 331), meta=np.ndarray>
    lat_psi         (eta_psi, xi_psi) float64 768kB dask.array<chunksize=(290, 331), meta=np.ndarray>
  * ocean_time      (ocean_time) datetime64[ns] 8B 2024-05-06T01:00:00
Dimensions without coordinates: tracer, boundary, eta_rho, xi_rho, eta_u, xi_u,
                                eta_v, xi_v, eta_psi, xi_psi
Data variables: (12/97)
    ntimes          int32 4B ...
    ndtfast         int32 4B ...
    dt              float64 8B ...
    dtfast          float64 8B ...
    dstart          datetime64[ns] 8B ...
    nHIS            int32 4B ...
    ...              ...
    temp            (ocean_time, s_rho, eta_rho, xi_rho) float32 8MB dask.array<chunksize=(1, 10, 146, 166), meta=np.ndarray>
    salt            (ocean_time, s_rho, eta_rho, xi_rho) float32 8MB dask.array<chunksize=(1, 10, 146, 166), meta=np.ndarray>
    oxygen          (ocean_time, s_rho, eta_rho, xi_rho) float32 8MB dask.array<chunksize=(1, 10, 146, 166), meta=np.ndarray>
    Pair            (ocean_time, eta_rho, xi_rho) float32 386kB dask.array<chunksize=(1, 291, 332), meta=np.ndarray>
    Uwind           (ocean_time, eta_rho, xi_rho) float32 386kB dask.array<chunksize=(1, 291, 332), meta=np.ndarray>
    Vwind           (ocean_time, eta_rho, xi_rho) float32 386kB dask.array<chunksize=(1, 291, 332), meta=np.ndarray>
Attributes: (12/36)
    file:              nos.cbofs.fields.forecast.20240506.t00z_0002.nc
    format:            netCDF-4/HDF5 file
    Conventions:       CF-1.4, SGRID-0.3
    type:              ROMS/TOMS history file
    title:             cbofs forecast RUN in operational mode
    var_info:          varinfo.yaml
    ...                ...
    compiler_flags:    -fp-model precise -ip -O3
    tiling:            008x016
    history:           ROMS/TOMS, Version 4.1, Monday - May 6, 2024 - 12:53:5...
    ana_file:          ROMS/Functionals/ana_btflux.h, ROMS/Functionals/ana_ra...
    CPP_options:       mode, ADD_FSOBC, ADD_M2OBC, ANA_BPFLUX, ANA_BSFLUX, AN...
    bio_file:          ROMS/Nonlinear/Biology/hypoxia_srm.h

In [4]:
%%time
len(ds.lat_rho.values)

CPU times: user 118 ms, sys: 34.5 ms, total: 153 ms
Wall time: 1.21 s


291

In [5]:
%%time
len(ds.lat_rho.values)

CPU times: user 2.87 ms, sys: 1.94 ms, total: 4.81 ms
Wall time: 16.4 ms


291